In [1]:
!nvidia-smi # checa #GPU

Mon May 29 03:40:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# permite interação com o filesystem do colab
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
!pip install ultralytics # instala o ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.7/592.7 kB 41.0 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO
 
model = YOLO("yolov8n.pt")  # importa um modelo pré-treinado

100%|██████████| 6.23M/6.23M [00:00<00:00, 294MB/s]


In [11]:
%mkdir "datasets" # cria uma pasta de 'datasets' para obedecer à estrutura de arquivos que o yolo espera

In [12]:
%cd {HOME}/datasets # vai para pasta de datasets

/content/datasets


In [13]:
# faz um override do encoding para evitar erros ao rodar o roboflow
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [14]:
# instala o roboflow e baixa dataset de rachaduras na pasta de datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="VRNu18b5353lvLkHicrX")
project = rf.workspace("university-bswxt").project("crack-bphdr")
dataset = project.version(2).download("yolov8")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.110, to fix: `pip install ultralytics<=8.0.20`

Extracting Dataset Version Zip to crack-2 in yolov8:: 100%|██████████| 8070/8070 [00:02<00:00, 3274.85it/s]


In [15]:
%cd {HOME} # volta para pasta raiz

/content


In [16]:
model.train(data="crack-2/data.yaml", epochs=10)  # treina modelo

Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=crack-2/data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, s

In [18]:
model.val()  # avalia modelo

Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /content/datasets/crack-2/valid/labels.cache... 200 images, 1 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:08<00:00,  1.59it/s]
                   all        200        249      0.862      0.707      0.788      0.544
Speed: 3.8ms preprocess, 6.6ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to runs/detect/val


ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x7fc0701e9f00>
fitness: 0.5687859533067601
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.54443])
names: {0: 'crack'}
plot: True
results_dict: {'metrics/precision(B)': 0.8622382606742421, 'metrics/recall(B)': 0.7068273092369478, 'metrics/mAP50(B)': 0.7880245741788879, 'metrics/mAP50-95(B)': 0.5444261065431903, 'fitness': 0.5687859533067601}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 3.8251757621765137, 'inference': 6.625492572784424, 'loss': 0.0017130374908447266, 'postprocess': 3.88957142829895}

In [17]:
model.export(format="onnx")  # exporta modelo; a partir daqui, podemos baixar o best.pt e utilizá-lo em nosso script python

Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients

PyTorch: starting from runs/detect/train2/weights/best.pt with input shape (16, 3, 640, 640) BCHW and output shape(s) (16, 5, 8400) (6.0 MB)
requirements: Ultralytics requirement "onnx>=1.12.0" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 323.6 MB/s eta 0:00:00

requirements: 1 package updated per ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.14.0 opset 17...
ONNX: export success ✅ 13.0s, saved as runs/detect/train2/weights/best.onnx (11.7 MB)

Export complete (21.3s)
Results saved to /content/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best.onnx imgsz=640 
Validate:     

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'runs/detect/train2/weights/best.onnx'